# FMLIKH Model: Creep Response

## Learning Objectives

1. Fit FMLIKH to **creep (step stress)** experimental data
2. Analyze **multi-mode delayed yielding** with distributed time scales
3. Compare FMLIKH vs single-mode FIKH creep predictions

## Prerequisites

- NB07: FMLIKH Flow Curve
- NB04: FIKH Creep (single-mode concepts)

## 1. Setup

In [ ]:
import sys
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"

In [ ]:
%matplotlib inline
import os, sys, time, warnings
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.fikh import FMLIKH

# Robust path resolution for execution from any directory
from pathlib import Path
_nb_dir = Path(__file__).parent if "__file__" in dir() else Path.cwd()
_utils_candidates = [_nb_dir / ".." / "utils", Path("examples/utils"), _nb_dir.parent / "utils"]
for _p in _utils_candidates:
    if (_p / "fikh_tutorial_utils.py").exists():
        sys.path.insert(0, str(_p.resolve()))
        break
from fikh_tutorial_utils import (
    load_ml_ikh_creep, save_fikh_results, print_convergence_summary,
    compute_fit_quality, get_fmlikh_param_names,
)

jax, jnp = safe_import_jax()
verify_float64()

# ============================================================
# FAST_MODE Configuration
# ============================================================
# True  = quick validation (~5-10 min, reduced samples)
# False = full Bayesian run (may need 30+ min on CPU)
FAST_MODE = True

if FAST_MODE:
    print("FAST_MODE: reduced samples for quick validation")
    NUM_WARMUP = 50
    NUM_SAMPLES = 100
    NUM_CHAINS = 1
else:
    print("FULL mode: complete Bayesian inference")
    NUM_WARMUP = 200
    NUM_SAMPLES = 500
    NUM_CHAINS = 1

## 2. Load Data

In [ ]:
STRESS_PAIR = 0
time_data, gamma_dot_data, sigma_init, sigma_final = load_ml_ikh_creep(stress_pair_index=STRESS_PAIR)

strain_data = np.cumsum(gamma_dot_data[:-1] * np.diff(time_data))
strain_data = np.insert(strain_data, 0, 0.0)

print(f"Stress: {sigma_final} Pa")
print(f"Data points: {len(time_data)}")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(time_data, strain_data, "ko", markersize=5)
ax.set_xlabel("Time [s]")
ax.set_ylabel("Strain [-]")
ax.set_title(f"Creep Data (σ = {sigma_final} Pa)")
ax.grid(True, alpha=0.3)
plt.tight_layout()
display(fig)
plt.close(fig)

## 3. NLSQ Fitting

In [ ]:
N_MODES = 3
model = FMLIKH(n_modes=N_MODES, include_thermal=False, shared_alpha=True, alpha_structure=0.7)

# For creep: we fit to strain vs time, but model needs stress history
# Create constant stress array
stress_data = np.full_like(time_data, sigma_final)

t0 = time.time()
model.fit(time_data, stress_data, test_mode="startup", strain=strain_data, method='scipy')
print(f"NLSQ time: {time.time() - t0:.2f} s")

stress_pred = model.predict(time_data, test_mode="startup", strain=strain_data)
metrics = compute_fit_quality(stress_data, stress_pred)
print(f"R² = {metrics['R2']:.6f}")

In [ ]:
time_fine = np.linspace(time_data.min(), time_data.max(), 300)
# Interpolate strain for fine time points
strain_fine = np.interp(time_fine, time_data, strain_data)
stress_pred_fine = model.predict(time_fine, test_mode="startup", strain=strain_fine)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(time_data, stress_data, "ko", markersize=5, alpha=0.7, label="Data")
ax.plot(time_fine, stress_pred_fine, "-", lw=2.5, color="C0", label=f"FMLIKH ({N_MODES} modes)")
ax.set_xlabel("Time [s]")
ax.set_ylabel("Stress [Pa]")
ax.set_title(f"FMLIKH Creep Fit (R² = {metrics['R2']:.5f})")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
display(fig)
plt.close(fig)

## 4. Bayesian Inference

In [ ]:
param_names = get_fmlikh_param_names(n_modes=N_MODES, shared_alpha=True)
initial_values = {n: model.parameters.get_value(n) for n in param_names if n in model.parameters}

print(f"Running NUTS: {NUM_WARMUP} warmup + {NUM_SAMPLES} samples x {NUM_CHAINS} chain(s)")
print(f"  FAST_MODE={FAST_MODE}, data points={len(time_data)}")

t0 = time.time()
result = model.fit_bayesian(
    time_data, stress_data, test_mode="startup", strain=strain_data,
    num_warmup=NUM_WARMUP, num_samples=NUM_SAMPLES, num_chains=NUM_CHAINS,
    initial_values=initial_values, seed=42,
)
print(f"Bayesian time: {time.time() - t0:.1f} s")

In [ ]:
key_params = ["G_0", "alpha_structure", "sigma_y0"]
print_convergence_summary(result, key_params)

## 5. Save Results

In [ ]:
save_fikh_results(model, result, "fmlikh", "creep", param_names)

## Key Takeaways

1. **Multi-mode creep** shows distributed yielding time scales
2. **Each mode contributes** to overall compliance
3. **Fractional memory** modifies approach to steady flow